In [ ]:
from robovast.common.analysis import read_output_files, read_output_csv, get_behavior_info
import pandas as pd
from robovast_nav.gui import MapVisualizer
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
DATA_DIR = ''

try:
    df = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "poses.csv"))

    # Generate colors for each test
    tests = df['test'].unique()
    colors = cm.rainbow(np.linspace(0, 1, len(tests)))

    df_behaviors = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "behaviors.csv"))
    df_behavior_info = get_behavior_info('differential_drive_robot.nav_through_poses', df_behaviors)

except Exception as e:
  print(f"Error reading rosbag files: {e}")
  raise SystemExit("No rosbag data found.")

## Overview

In [ ]:
# Multi-Run Comparison: All Robot Paths Ground Truth

# Create single visualization with all robot paths ground truth
viz = MapVisualizer()
map_file = df['map_file'].iloc[0]
test_num = df['test'].iloc[0]
viz.load_map(os.path.join(DATA_DIR, test_num, map_file))
viz.create_figure(figsize=(14, 12))

# Draw all robot paths
for test, color in zip(tests, colors):
    df_test = df[df['test'] == test]
    df_gt_mask = df_test['frame'] == 'nav2_turtlebot4_base_link_gt'
    path_robot = list(zip(df_test.loc[df_gt_mask, 'position.x'], df_test.loc[df_gt_mask, 'position.y']))
    viz.draw_path(path_robot, color=color, linewidth=1.5, label=f'Test {test}', show_endpoints=False)

viz.ax.set_title(f'Multi-Run Comparison: All {len(tests)} Robot Paths', 
                 fontsize=14, fontweight='bold')
viz.ax.legend(loc='upper left', fontsize=9, ncol=2)

plt.tight_layout()
plt.show()

## Navigation Duration Analysis

In [ ]:
# Navigation Duration Comparison - Crisp Visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Prepare data
durations = df_behavior_info['duration'].values
tests_behavior = df_behavior_info['test'].values
x_pos = np.arange(len(durations))

# 1. Bar Chart - Duration by Run
ax = axes[0, 0]
bars = ax.bar(x_pos, durations, alpha=0.75, color=colors, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Run Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Navigation Duration by Run', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([f'{t}' for t in tests_behavior])
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# Add value labels
for i, duration in enumerate(durations):
    ax.text(i, duration + 0.5, f'{duration:.1f}s', ha='center', va='bottom', 
            fontsize=9, fontweight='bold')

# Add mean line
mean_duration = durations.mean()
ax.axhline(mean_duration, color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {mean_duration:.1f}s', alpha=0.7)
ax.legend(fontsize=10)

# 2. Box Plot - Duration Distribution
ax = axes[0, 1]
bp = ax.boxplot([durations], widths=0.5, patch_artist=True, 
                tick_labels=['All Runs'], showmeans=True)
bp['boxes'][0].set_facecolor('steelblue')
bp['boxes'][0].set_alpha(0.7)
bp['boxes'][0].set_edgecolor('black')
bp['boxes'][0].set_linewidth(2)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=2)
plt.setp(bp['means'], marker='D', markerfacecolor='red', markeredgecolor='black', markersize=8)

ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Navigation Duration Distribution', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# Add statistics annotations
q1, median, q3 = np.percentile(durations, [25, 50, 75])
ax.text(1.3, q1, f'Q1: {q1:.1f}s', fontsize=9, va='center')
ax.text(1.3, median, f'Median: {median:.1f}s', fontsize=9, va='center', fontweight='bold')
ax.text(1.3, q3, f'Q3: {q3:.1f}s', fontsize=9, va='center')
ax.text(1.3, mean_duration, f'Mean: {mean_duration:.1f}s', fontsize=9, va='center', color='red')

# 3. Histogram - Duration Distribution
ax = axes[0, 2]
n, bins, patches = ax.hist(durations, bins=15, color='steelblue', alpha=0.7, 
                           edgecolor='black', linewidth=1.5)

# Color bars based on value
for i, patch in enumerate(patches):
    patch.set_facecolor(colors[i % len(colors)])

ax.axvline(mean_duration, color='red', linestyle='--', linewidth=2.5, 
           label=f'Mean: {mean_duration:.1f}s', alpha=0.8)
ax.axvline(median, color='green', linestyle='--', linewidth=2.5, 
           label=f'Median: {median:.1f}s', alpha=0.8)
ax.set_xlabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax.set_title('Duration Distribution', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# 4. Sorted Duration Plot
ax = axes[1, 0]
sorted_indices = np.argsort(durations)
sorted_durations = durations[sorted_indices]
sorted_tests = tests_behavior[sorted_indices]

bars = ax.bar(range(len(sorted_durations)), sorted_durations, alpha=0.75, 
              color=[colors[i] for i in sorted_indices], edgecolor='black', linewidth=1.5)
ax.set_xlabel('Sorted Run Index', fontsize=11, fontweight='bold')
ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Navigation Duration (Sorted)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# Add labels for best and worst
ax.text(0, sorted_durations[0] + 0.5, f'Best: {sorted_durations[0]:.1f}s\n(Run {sorted_tests[0]})', 
        ha='center', va='bottom', fontsize=9, fontweight='bold', color='green')
ax.text(len(sorted_durations)-1, sorted_durations[-1] + 0.5, 
        f'Worst: {sorted_durations[-1]:.1f}s\n(Run {sorted_tests[-1]})', 
        ha='center', va='bottom', fontsize=9, fontweight='bold', color='red')

# 5. Cumulative Distribution
ax = axes[1, 1]
sorted_durations_cdf = np.sort(durations)
cumulative = np.arange(1, len(sorted_durations_cdf) + 1) * (100.0 / len(sorted_durations_cdf))

ax.plot(sorted_durations_cdf, cumulative, color='steelblue', linewidth=3, marker='o', 
        markersize=8, markerfacecolor='white', markeredgecolor='steelblue', markeredgewidth=2)
ax.set_xlabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_ylabel('Cumulative Percentage (%)', fontsize=11, fontweight='bold')
ax.set_title('Cumulative Distribution of Duration', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, linewidth=0.5)

# Add reference lines
for percentile in [25, 50, 75]:
    value = np.percentile(durations, percentile)
    ax.axhline(percentile, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax.axvline(value, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax.text(value, percentile + 2, f'{percentile}%: {value:.1f}s', fontsize=9, fontweight='bold')

# 6. Deviation from Mean
ax = axes[1, 2]
deviations = durations - mean_duration
colors_dev = ['green' if d <= 0 else 'red' for d in deviations]

bars = ax.bar(x_pos, deviations, alpha=0.75, color=colors_dev, edgecolor='black', linewidth=1.5)
ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.set_xlabel('Run Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Deviation from Mean (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Duration Deviation from Mean', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([f'{t}' for t in tests_behavior])
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# Add value labels
for i, dev in enumerate(deviations):
    va = 'bottom' if dev > 0 else 'top'
    offset = 0.2 if dev > 0 else -0.2
    ax.text(i, dev + offset, f'{dev:+.1f}s', ha='center', va=va, 
            fontsize=8, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for multi-run comparison
import numpy as np
import matplotlib.pyplot as plt

# Split by frame and prepare for all runs
df_robot = df[df['frame'] == 'base_link'].copy()
df_groundtruth = df[df['frame'] == 'nav2_turtlebot4_base_link_gt'].copy()

# Sort by test and timestamp
df_robot.sort_values(['test', 'timestamp'], inplace=True)
df_groundtruth.sort_values(['test', 'timestamp'], inplace=True)

# print(f"Processing {len(tests)} runs for comparison")
# print(f"Robot data points: {len(df_robot)}")
# print(f"Ground truth data points: {len(df_groundtruth)}")

In [ ]:
# Calculate errors for each run separately
from scipy.interpolate import interp1d

# Store results for each run
run_errors = {}
run_stats = []

for test in tests:
    # Get data for this specific run
    df_robot_test = df_robot[df_robot['test'] == test].copy()
    df_gt_test = df_groundtruth[df_groundtruth['test'] == test].copy()
    if df_robot_test.empty or df_gt_test.empty:
        print(f"Skipping test {test} due to empty data")
        continue
    # Find common time range
    time_start = max(df_robot_test['timestamp'].iloc[0], df_gt_test['timestamp'].iloc[0])
    time_end = min(df_robot_test['timestamp'].iloc[-1], df_gt_test['timestamp'].iloc[-1])
    
    # Create interpolation functions for ground truth
    gt_timestamps = df_gt_test['timestamp'].values
    gt_interp_x = interp1d(gt_timestamps, df_gt_test['position.x'].values, kind='linear', fill_value='extrapolate')
    gt_interp_y = interp1d(gt_timestamps, df_gt_test['position.y'].values, kind='linear', fill_value='extrapolate')
    gt_interp_yaw = interp1d(gt_timestamps, df_gt_test['orientation.yaw'].values, kind='linear', fill_value='extrapolate')
    
    # Filter robot data to common time range
    time_mask = (df_robot_test['timestamp'] >= time_start) & (df_robot_test['timestamp'] <= time_end)
    df_robot_aligned = df_robot_test[time_mask].copy()
    
    robot_timestamps = df_robot_aligned['timestamp'].values
    
    # Interpolate ground truth at robot timestamps
    gt_x = gt_interp_x(robot_timestamps)
    gt_y = gt_interp_y(robot_timestamps)
    gt_yaw = gt_interp_yaw(robot_timestamps)
    
    # Calculate position errors
    position_error_x = df_robot_aligned['position.x'].values - gt_x
    position_error_y = df_robot_aligned['position.y'].values - gt_y
    absolute_position_error = np.sqrt(position_error_x**2 + position_error_y**2)
    
    # Calculate orientation error
    orientation_error = df_robot_aligned['orientation.yaw'].values - gt_yaw
    orientation_error = np.arctan2(np.sin(orientation_error), np.cos(orientation_error))
    
    # Store errors for this run
    run_errors[test] = {
        'timestamps': robot_timestamps,
        'position_error': absolute_position_error,
        'position_error_x': position_error_x,
        'position_error_y': position_error_y,
        'orientation_error': orientation_error
    }
    
    # Calculate statistics
    run_stats.append({
        'test': test,
        'mean_pos_error': np.mean(absolute_position_error),
        'std_pos_error': np.std(absolute_position_error),
        'max_pos_error': np.max(absolute_position_error),
        'median_pos_error': np.median(absolute_position_error),
        'p95_pos_error': np.percentile(absolute_position_error, 95),
        'mean_orient_error': np.mean(np.abs(orientation_error)),
        'std_orient_error': np.std(orientation_error),
        'max_orient_error': np.max(np.abs(orientation_error))
    })

# Convert to DataFrame for easy analysis
stats_df = pd.DataFrame(run_stats)

In [ ]:
from robovast.common.analysis import calculate_speeds_from_poses
df_gt_speeds = calculate_speeds_from_poses(df_groundtruth)
df_gt_speeds

In [ ]:
# # Speed Statistics by Run
# print("Speed Statistics by Test Run:")
# print("="*80)

speed_stats = []
for test in tests:
    df_test_speeds = df_gt_speeds[df_gt_speeds['test'] == test]
    
    linear_speeds = df_test_speeds['linear_speed'].values
    angular_speeds = df_test_speeds['angular_speed'].values

    if len(linear_speeds) == 0 or len(angular_speeds) == 0:
        print(f"Skipping speed statistics for test {test} due to empty speed data")
        continue
    
    speed_stats.append({
        'test': test,
        'mean_linear': np.mean(linear_speeds),
        'max_linear': np.max(linear_speeds),
        'std_linear': np.std(linear_speeds),
        'median_linear': np.median(linear_speeds),
        'mean_angular': np.mean(np.abs(angular_speeds)),
        'max_angular': np.max(np.abs(angular_speeds)),
        'std_angular': np.std(angular_speeds),
        'median_angular': np.median(np.abs(angular_speeds))
    })

speed_stats_df = pd.DataFrame(speed_stats)
# print(speed_stats_df)
# print("\nOverall Statistics:")
# print(f"Mean Linear Speed (avg): {speed_stats_df['mean_linear'].mean():.3f} m/s (±{speed_stats_df['mean_linear'].std():.3f})")
# print(f"Mean Angular Speed (avg): {speed_stats_df['mean_angular'].mean():.3f} rad/s (±{speed_stats_df['mean_angular'].std():.3f})")

In [ ]:
# Duration vs Performance Metrics Comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Duration vs Mean Position Error
ax = axes[0]
durations_for_scatter = df_behavior_info['duration'].values
if 'mean_pos_error' in stats_df:
    mean_pos_errors = stats_df['mean_pos_error'].values

    scatter = ax.scatter(durations_for_scatter, mean_pos_errors, c=colors, s=250, 
                        alpha=0.75, edgecolors='black', linewidth=2)

    # Add labels for each point
    for i, test in enumerate(tests):
        ax.annotate(f'Run {test}', (durations_for_scatter[i], mean_pos_errors[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

    # Add trend line only if we have enough data points
    if len(durations_for_scatter) > 1:
        with np.errstate(all='ignore'):  # Suppress warnings for single data point
            z = np.polyfit(durations_for_scatter, mean_pos_errors, 1)
            p = np.poly1d(z)
            ax.plot(durations_for_scatter, p(durations_for_scatter), "r--", alpha=0.8, linewidth=2,
                    label=f'Trend: y={z[0]:.4f}x+{z[1]:.4f}')
            ax.legend(fontsize=10)

    ax.set_xlabel('Navigation Duration (seconds)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Mean Position Error (m)', fontsize=12, fontweight='bold')
    ax.set_title('Duration vs Position Error', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, linewidth=0.5)

    # Calculate correlation only if we have enough data points
    if len(durations_for_scatter) > 1:
        with np.errstate(all='ignore'):  # Suppress warnings
            correlation = np.corrcoef(durations_for_scatter, mean_pos_errors)[0, 1]
            if not np.isnan(correlation):
                ax.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                        transform=ax.transAxes, fontsize=11, fontweight='bold',
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7), verticalalignment='top')
    else:
        ax.text(0.5, 0.05, 'Need multiple runs for correlation analysis', 
                transform=ax.transAxes, fontsize=10, ha='center', style='italic')

# 2. Duration vs Mean Linear Speed
ax = axes[1]
if 'mean_linear' in speed_stats_df:
    mean_linear_speeds = speed_stats_df['mean_linear'].values

    scatter = ax.scatter(durations_for_scatter, mean_linear_speeds, c=colors, s=250, 
                        alpha=0.75, edgecolors='black', linewidth=2)

    # Add labels
    for i, test in enumerate(tests):
        ax.annotate(f'Run {test}', (durations_for_scatter[i], mean_linear_speeds[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

    # Add trend line only if we have enough data points
    if len(durations_for_scatter) > 1:
        with np.errstate(all='ignore'):  # Suppress warnings for single data point
            z = np.polyfit(durations_for_scatter, mean_linear_speeds, 1)
            p = np.poly1d(z)
            ax.plot(durations_for_scatter, p(durations_for_scatter), "r--", alpha=0.8, linewidth=2,
                    label=f'Trend: y={z[0]:.4f}x+{z[1]:.4f}')
            ax.legend(fontsize=10)

    ax.set_xlabel('Navigation Duration (seconds)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Mean Linear Speed (m/s)', fontsize=12, fontweight='bold')
    ax.set_title('Duration vs Mean Speed', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, linewidth=0.5)

    # Calculate correlation only if we have enough data points
    if len(durations_for_scatter) > 1:
        with np.errstate(all='ignore'):  # Suppress warnings
            correlation = np.corrcoef(durations_for_scatter, mean_linear_speeds)[0, 1]
            if not np.isnan(correlation):
                ax.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                        transform=ax.transAxes, fontsize=11, fontweight='bold',
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7), verticalalignment='top')
    else:
        ax.text(0.5, 0.05, 'Need multiple runs for correlation analysis', 
                transform=ax.transAxes, fontsize=10, ha='center', style='italic')

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*80)
print("DURATION ANALYSIS SUMMARY")
print("="*80)
print(f"\nNavigation Duration:")
print(f"  Mean: {mean_duration:.2f} seconds")
print(f"  Std Dev: {durations.std():.2f} seconds")
print(f"  Range: {durations.min():.2f}s - {durations.max():.2f}s ({durations.max() - durations.min():.2f}s difference)")
print(f"  Coefficient of Variation: {(durations.std()/mean_duration)*100:.2f}%")
print(f"\nBest Run: {sorted_tests[0]} with {sorted_durations[0]:.2f} seconds")
print(f"Worst Run: {sorted_tests[-1]} with {sorted_durations[-1]:.2f} seconds")
print(f"Performance Difference: {((sorted_durations[-1] - sorted_durations[0]) / sorted_durations[0] * 100):.1f}%")
print("="*80)

## Localization Errors

In [ ]:
error_matrix = []
if test in run_errors:
    # Multi-Run Comparison: Position Error Over Time
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. Position errors over time for all runs
    ax = axes[0, 0]
    for test, color in zip(tests, colors):
        errors = run_errors[test]
        # Normalize time to start at 0
        time_normalized = errors['timestamps'] - errors['timestamps'][0]
        ax.plot(time_normalized, errors['position_error'], color=color, alpha=0.6, linewidth=1.5, label=f'Run {test}')

    ax.set_xlabel('Time (s)', fontsize=11)
    ax.set_ylabel('Absolute Position Error (m)', fontsize=11)
    ax.set_title('Position Error Over Time - All Runs', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3)

    # 2. Mean error trajectory with standard deviation band
    ax = axes[0, 1]
    # Find common time length (use shortest run)
    min_length = min(len(run_errors[test]['position_error']) for test in tests)

    # Interpolate all runs to common time grid
    common_time = np.linspace(0, 100, min_length)  # Normalized to 100 seconds

    for test in tests:
        errors = run_errors[test]
        time_normalized = np.linspace(0, 100, len(errors['position_error']))
        interp_func = interp1d(time_normalized, errors['position_error'], kind='linear')
        error_matrix.append(interp_func(common_time))

    error_matrix = np.array(error_matrix)
    mean_error = np.mean(error_matrix, axis=0)
    std_error = np.std(error_matrix, axis=0)

    ax.plot(common_time, mean_error, 'b-', linewidth=2.5, label='Mean')
    ax.fill_between(common_time, mean_error - std_error, mean_error + std_error, 
                    alpha=0.3, color='blue', label='±1 Std Dev')
    ax.set_xlabel('Normalized Time (s)', fontsize=11)
    ax.set_ylabel('Absolute Position Error (m)', fontsize=11)
    ax.set_title('Mean Position Error with Variability Band', fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

    # 3. Orientation errors over time for all runs
    ax = axes[1, 0]
    for test, color in zip(tests, colors):
        errors = run_errors[test]
        time_normalized = errors['timestamps'] - errors['timestamps'][0]
        ax.plot(time_normalized, np.abs(errors['orientation_error']), color=color, alpha=0.6, linewidth=1.5, label=f'Run {test}')

    ax.set_xlabel('Time (s)', fontsize=11)
    ax.set_ylabel('Absolute Orientation Error (rad)', fontsize=11)
    ax.set_title('Orientation Error Over Time - All Runs', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3)

    # 4. Box plot comparison of position errors across runs
    ax = axes[1, 1]
    error_data = [run_errors[test]['position_error'] for test in tests]
    bp = ax.boxplot(error_data, tick_labels=[f'{t}' for t in tests], patch_artist=True)

    # Color boxes
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)

    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Absolute Position Error (m)', fontsize=11)
    ax.set_title('Position Error Distribution Across Runs', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

In [ ]:
# Statistical Comparison Across Runs
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Bar chart of mean errors with error bars (std dev)
ax = axes[0, 0]
x_pos = np.arange(len(tests))
if 'mean_pos_error' in stats_df and 'std_pos_error' in stats_df:
    means = stats_df['mean_pos_error'].values
    stds = stats_df['std_pos_error'].values

    bars = ax.bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Mean Position Error (m)', fontsize=11)
    ax.set_title('Mean Position Error by Run (with Std Dev)', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for i, (mean, std) in enumerate(zip(means, stds)):
        ax.text(i, mean + std + 0.01, f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

# 2. Max error comparison
ax = axes[0, 1]
if 'max_pos_error' in stats_df:
    maxs = stats_df['max_pos_error'].values
    bars = ax.bar(x_pos, maxs, alpha=0.7, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Max Position Error (m)', fontsize=11)
    ax.set_title('Maximum Position Error by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y')

# 3. Mean orientation error comparison
ax = axes[1, 0]
if 'mean_orient_error' in stats_df and 'std_orient_error' in stats_df:
    orient_means = stats_df['mean_orient_error'].values
    orient_stds = stats_df['std_orient_error'].values

    bars = ax.bar(x_pos, orient_means, yerr=orient_stds, capsize=5, alpha=0.7, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Mean Orientation Error (rad)', fontsize=11)
    ax.set_title('Mean Orientation Error by Run (with Std Dev)', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y')

# 4. Percentile comparison (50th, 95th)
ax = axes[1, 1]
if 'median_pos_error' in stats_df and 'p95_pos_error' in stats_df:
    p50s = stats_df['median_pos_error'].values
    p95s = stats_df['p95_pos_error'].values

    width = 0.35
    x_pos1 = x_pos - width/2
    x_pos2 = x_pos + width/2

    ax.bar(x_pos1, p50s, width, label='Median (50th)', alpha=0.7, color='skyblue', edgecolor='black', linewidth=1.5)
    ax.bar(x_pos2, p95s, width, label='95th Percentile', alpha=0.7, color='coral', edgecolor='black', linewidth=1.5)

    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Position Error (m)', fontsize=11)
    ax.set_title('Position Error Percentiles by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
if test in run_errors:
    # Error Distribution Comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # 1. Overlayed histograms of position errors
    ax = axes[0, 0]
    for test, color in zip(tests, colors):
        errors = run_errors[test]['position_error']
        ax.hist(errors, bins=40, alpha=0.5, color=color, label=f'Run {test}', edgecolor='black', linewidth=0.5)

    ax.set_xlabel('Absolute Position Error (m)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title('Position Error Distribution - All Runs Overlayed', fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3, axis='y')

    # 2. Cumulative distribution functions for all runs
    ax = axes[0, 1]
    for test, color in zip(tests, colors):
        errors = run_errors[test]['position_error']
        sorted_errors = np.sort(errors)
        cumulative = np.arange(1, len(sorted_errors) + 1) * (100.0 / len(sorted_errors))
        ax.plot(sorted_errors, cumulative, color=color, linewidth=2, alpha=0.7, label=f'Run {test}')

    ax.set_xlabel('Absolute Position Error (m)', fontsize=11)
    ax.set_ylabel('Cumulative Percentage (%)', fontsize=11)
    ax.set_title('Cumulative Distribution of Position Errors', fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3)

    # Add reference lines for key percentiles
    ax.axhline(50, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax.axhline(95, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    ax.text(ax.get_xlim()[1]*0.95, 50, '50%', ha='right', va='bottom', fontsize=9)
    ax.text(ax.get_xlim()[1]*0.95, 95, '95%', ha='right', va='bottom', fontsize=9)

    # 3. Violin plots showing distribution shape
    ax = axes[1, 0]
    error_data = [run_errors[test]['position_error'] for test in tests]
    parts = ax.violinplot(error_data, positions=range(len(tests)), showmeans=True, showmedians=True)

    # Color the violin plots
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors[i])
        pc.set_alpha(0.6)

    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Absolute Position Error (m)', fontsize=11)
    ax.set_title('Position Error Distribution Shape by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(range(len(tests)))
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y')

    # 4. Orientation error distributions (overlayed histograms)
    ax = axes[1, 1]
    for test, color in zip(tests, colors):
        errors = np.abs(run_errors[test]['orientation_error'])
        ax.hist(errors, bins=40, alpha=0.5, color=color, label=f'Run {test}', edgecolor='black', linewidth=0.5)

    ax.set_xlabel('Absolute Orientation Error (rad)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title('Orientation Error Distribution - All Runs Overlayed', fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

In [ ]:
# Aggregate Statistics and Summary
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Summary statistics table visualization
ax = axes[0, 0]
ax.axis('tight')
ax.axis('off')

if 'mean_pos_error' in stats_df:
    # Calculate overall statistics
    overall_mean = stats_df['mean_pos_error'].mean()
    overall_std = stats_df['mean_pos_error'].std()
    overall_min = stats_df['mean_pos_error'].min()
    overall_max = stats_df['mean_pos_error'].max()

    table_data = [
        ['Metric', 'Value'],
        ['Overall Mean Pos Error', f'{overall_mean:.4f} m'],
        ['Std Dev Across Runs', f'{overall_std:.4f} m'],
        ['Best Run Mean Error', f'{overall_min:.4f} m'],
        ['Worst Run Mean Error', f'{overall_max:.4f} m'],
        ['Coefficient of Variation', f'{(overall_std/overall_mean)*100:.2f}%'],
        ['', ''],
        ['Mean Orient Error (avg)', f'{stats_df["mean_orient_error"].mean():.4f} rad ({np.degrees(stats_df["mean_orient_error"].mean()):.2f}°)'],
        ['Orient Error Std Dev', f'{stats_df["mean_orient_error"].std():.4f} rad ({np.degrees(stats_df["mean_orient_error"].std()):.2f}°)'],
    ]

    table = ax.table(cellText=table_data, cellLoc='left', loc='center',
                    colWidths=[0.5, 0.5])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2.5)

    # Style header row
    for i in range(2):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')

    ax.set_title('Overall Statistics Across All Runs', fontsize=12, fontweight='bold', pad=20)
else:
    overall_mean = None
    overall_std = None
    overall_min = None
    overall_max = None

# 2. Scatter plot: Mean vs Std Dev for each run
ax = axes[0, 1]
if 'mean_pos_error' in stats_df and 'std_pos_error' in stats_df:
    scatter = ax.scatter(stats_df['mean_pos_error'], stats_df['std_pos_error'], 
                        c=colors, s=200, alpha=0.7, edgecolors='black', linewidth=2)

    for i, test in enumerate(tests):
        ax.annotate(f'Run {test}', 
                (stats_df.iloc[i]['mean_pos_error'], stats_df.iloc[i]['std_pos_error']),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

    ax.set_xlabel('Mean Position Error (m)', fontsize=11)
    ax.set_ylabel('Standard Deviation (m)', fontsize=11)
    ax.set_title('Mean vs Variability by Run', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

# 3. Run consistency heatmap (correlation between runs)
ax = axes[1, 0]

# Create correlation matrix between runs (using interpolated errors)
if error_matrix is not None and error_matrix.size > 0 and len(error_matrix) > 1:
    correlation_matrix = np.corrcoef(error_matrix)
    
    im = ax.imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=0, vmax=1)
    ax.set_xticks(range(len(tests)))
    ax.set_yticks(range(len(tests)))
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.set_yticklabels([f'{t}' for t in tests])
    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Run Number', fontsize=11)
    ax.set_title('Error Pattern Correlation Between Runs', fontsize=12, fontweight='bold')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Correlation', fontsize=10)
    
    # Add correlation values
    for i in range(len(tests)):
        for j in range(len(tests)):
            text = ax.text(j, i, f'{correlation_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black", fontsize=8)
else:
    ax.text(0.5, 0.5, 'Insufficient data for correlation analysis\n(need at least 2 runs)', 
            ha='center', va='center', transform=ax.transAxes, fontsize=11)
    ax.set_title('Error Pattern Correlation Between Runs', fontsize=12, fontweight='bold')
    ax.axis('off')

# 4. Range and quartile visualization
ax = axes[1, 1]
if test in run_errors:
    # Calculate quartiles for each run
    quartile_data = []
    for test in tests:
        errors = run_errors[test]['position_error']
        q1 = np.percentile(errors, 25)
        q2 = np.percentile(errors, 50)
        q3 = np.percentile(errors, 75)
        min_val = np.min(errors)
        max_val = np.max(errors)
        quartile_data.append([min_val, q1, q2, q3, max_val])

    quartile_data = np.array(quartile_data)
    x_pos = np.arange(len(tests))

    # Plot ranges
    for i, (color, data) in enumerate(zip(colors, quartile_data)):
        ax.plot([i, i], [data[0], data[4]], color=color, linewidth=2, alpha=0.5)
        ax.plot([i, i], [data[1], data[3]], color=color, linewidth=6, alpha=0.8)
        ax.scatter(i, data[2], color=color, s=100, zorder=5, edgecolors='black', linewidth=2)

    ax.set_xlabel('Run Number', fontsize=11)
    ax.set_ylabel('Position Error (m)', fontsize=11)
    ax.set_title('Error Range and Quartiles by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y')

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='gray', linewidth=2, alpha=0.5, label='Min-Max Range'),
    Line2D([0], [0], color='gray', linewidth=6, alpha=0.8, label='Q1-Q3 (IQR)'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', markersize=8, 
           markeredgecolor='black', markeredgewidth=2, label='Median')
]
ax.legend(handles=legend_elements, fontsize=9)

plt.tight_layout()
plt.show()

# Print comprehensive summary
print("\n" + "="*60)
print("COMPREHENSIVE MULTI-RUN COMPARISON SUMMARY")
print("="*60)
print(f"\nNumber of Runs: {len(tests)}")
print(f"\nPosition Error Statistics Across All Runs:")
if overall_mean:
    print(f"  Mean of Means: {overall_mean:.4f} m")
if overall_std:
    print(f"  Std Dev of Means: {overall_std:.4f} m")
if overall_mean and overall_std:
    print(f"  Coefficient of Variation: {(overall_std/overall_mean)*100:.2f}%")
if overall_min and overall_max:
    print(f"  Best Run: {stats_df.loc[stats_df['mean_pos_error'].idxmin(), 'test']} ({overall_min:.4f} m)")
    print(f"  Worst Run: {stats_df.loc[stats_df['mean_pos_error'].idxmax(), 'test']} ({overall_max:.4f} m)")
    print(f"  Range: {overall_max - overall_min:.4f} m")

## Speed Analysis

In [ ]:
# Multi-Run Speed Comparison - Crisp Visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Linear Speed Distribution Comparison
ax = axes[0, 0]
for test, color in zip(tests, colors):
    df_test_speeds = df_gt_speeds[df_gt_speeds['test'] == test]
    ax.hist(df_test_speeds['linear_speed'].values, bins=40, alpha=0.5, 
            color=color, label=f'Run {test}', edgecolor='black', linewidth=0.5)

ax.set_xlabel('Linear Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax.set_title('Linear Speed Distribution - All Runs', fontsize=12, fontweight='bold')
ax.legend(fontsize=8, ncol=2)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# 2. Angular Speed Distribution Comparison
ax = axes[0, 1]
for test, color in zip(tests, colors):
    df_test_speeds = df_gt_speeds[df_gt_speeds['test'] == test]
    ax.hist(df_test_speeds['angular_speed'].values, bins=40, alpha=0.5, 
            color=color, label=f'Run {test}', edgecolor='black', linewidth=0.5)

ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Angular Speed (rad/s)', fontsize=11, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax.set_title('Angular Speed Distribution - All Runs', fontsize=12, fontweight='bold')
ax.legend(fontsize=8, ncol=2)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# 3. Box Plot - Linear Speed Comparison
ax = axes[0, 2]
linear_speed_data = [df_gt_speeds[df_gt_speeds['test'] == test]['linear_speed'].values 
                     for test in tests]
bp = ax.boxplot(linear_speed_data, tick_labels=[f'{t}' for t in tests], 
                patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Run Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Linear Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_title('Linear Speed Distribution by Run', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# 4. Bar Chart - Mean Linear Speed with Error Bars
ax = axes[1, 0]
x_pos = np.arange(len(tests))
if 'mean_linear' in speed_stats_df and 'std_linear' in speed_stats_df:
    linear_means = speed_stats_df['mean_linear'].values
    linear_stds = speed_stats_df['std_linear'].values

    bars = ax.bar(x_pos, linear_means, yerr=linear_stds, capsize=5, 
                alpha=0.75, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Run Number', fontsize=11, fontweight='bold')
    ax.set_ylabel('Mean Linear Speed (m/s)', fontsize=11, fontweight='bold')
    ax.set_title('Mean Linear Speed by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

    # Add value labels
    for i, (mean, std) in enumerate(zip(linear_means, linear_stds)):
        ax.text(i, mean + std + 0.01, f'{mean:.3f}', ha='center', va='bottom', 
                fontsize=8, fontweight='bold')

# 5. Bar Chart - Mean Angular Speed with Error Bars
ax = axes[1, 1]
if 'mean_angular' in speed_stats_df and 'std_angular' in speed_stats_df:
    angular_means = speed_stats_df['mean_angular'].values
    angular_stds = speed_stats_df['std_angular'].values

    bars = ax.bar(x_pos, angular_means, yerr=angular_stds, capsize=5, 
                alpha=0.75, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Run Number', fontsize=11, fontweight='bold')
    ax.set_ylabel('Mean Angular Speed (rad/s)', fontsize=11, fontweight='bold')
    ax.set_title('Mean Angular Speed by Run', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'{t}' for t in tests])
    ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

    # Add value labels
    for i, (mean, std) in enumerate(zip(angular_means, angular_stds)):
        ax.text(i, mean + std + 0.01, f'{mean:.3f}', ha='center', va='bottom', 
                fontsize=8, fontweight='bold')

# 6. Scatter Plot - Max Speed Comparison
ax = axes[1, 2]
if 'max_linear' in speed_stats_df and 'max_angular' in speed_stats_df:
    max_linear = speed_stats_df['max_linear'].values
    max_angular = speed_stats_df['max_angular'].values

    scatter = ax.scatter(max_linear, max_angular, c=colors, s=250, alpha=0.75, 
                        edgecolors='black', linewidth=2)

    for i, test in enumerate(tests):
        ax.annotate(f'Run {test}', (max_linear[i], max_angular[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

    ax.set_xlabel('Max Linear Speed (m/s)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Max Angular Speed (rad/s)', fontsize=11, fontweight='bold')
    ax.set_title('Maximum Speed Comparison', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Speed Profile Over Time - Multi-Run Comparison
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# 1. Linear Speed Over Time - All Runs
ax = axes[0]
for test, color in zip(tests, colors):
    df_test_speeds = df_gt_speeds[df_gt_speeds['test'] == test]
    timestamps = df_test_speeds['timestamp'].values
    if len(timestamps) == 0:
        print(f"Warning: No speed data for test {test}. Skipping.")
        continue
    # Normalize time to start at 0
    timestamps_norm = timestamps - timestamps[0]
    linear_speeds = df_test_speeds['linear_speed'].values
    
    ax.plot(timestamps_norm, linear_speeds, color=color, alpha=0.7, 
            linewidth=1.5, label=f'Run {test}')

ax.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax.set_ylabel('Linear Speed (m/s)', fontsize=12, fontweight='bold')
ax.set_title('Linear Speed Profile Over Time - All Runs', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9, ncol=3)
ax.grid(True, alpha=0.3, linewidth=0.5)
ax.set_ylim(bottom=0)

# 2. Angular Speed Over Time - All Runs
ax = axes[1]
for test, color in zip(tests, colors):
    df_test_speeds = df_gt_speeds[df_gt_speeds['test'] == test]
    timestamps = df_test_speeds['timestamp'].values
    if len(timestamps) == 0:
        print(f"Warning: No speed data for test {test}. Skipping.")
        continue
    timestamps_norm = timestamps - timestamps[0]
    angular_speeds = df_test_speeds['angular_speed'].values
    
    ax.plot(timestamps_norm, angular_speeds, color=color, alpha=0.7, 
            linewidth=1.5, label=f'Run {test}')

ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax.set_ylabel('Angular Speed (rad/s)', fontsize=12, fontweight='bold')
ax.set_title('Angular Speed Profile Over Time - All Runs', fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9, ncol=3)
ax.grid(True, alpha=0.3, linewidth=0.5)

plt.tight_layout()
plt.show()